## Notebook to process and clean all controls of legislators

In [389]:
import pandas as pd
import re
import numpy as np
from os import listdir
import json
from tqdm import tqdm
import difflib 

In [165]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [174]:
def education_num(x):
    if x is None:
        return(0)
    elif x  == 'educacion basica':
        return(1)
    elif x in 'educacion media superior':
        return(2)
    elif x == 'educacion superior':
        return(3)
    else:
        return(0)

In [217]:
def education(x):
    if x in ['Primaria', 'Secundaria']:
        return('educacion basica')
    elif x in ['Nivel Medio Superior', 'Técnica', 'Comercial', 'Secundaria', 'Carrera Técnica', 
              'Carrera Trunca', 'Carrera Comercial', 'Bachillerato', 'Bachillerato | Nivel Medio Superior','Trunca', 
              'Trunco', 'Talleres', 'Taller', 'Pasante', 'Contador Privado', 'Conferencia', 'Curso', 'Otro', '']:
        return('educacion media superior')
    elif x in ['Carrera', 'Licenciatura','Doctorado', 'Maestría',
              'Posgrado', 'Especialidad', 'Estudios Superiores', 'Carrera', 'Estudios Profesionales' ,
              'Estudios de Maestría', 'Postgrado', 'Doctorante', 'Licenciatura (cursando actualmente)', 'Estudios', 
              'Profesor Normalista', 'Profesor', '2', '5', '3', '1', '4', 'Diplomado', 'Seminario']:
        return('educacion superior')
        

## ID-Data

In [193]:
id_data = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-collection_data/00-id_data/00-id/id_data_aggregate.xlsx')

In [194]:
id_propietarios = id_data[id_data['suplente_propietario'] == 'P']

In [195]:
len(id_propietarios.id_legislador.unique())

2500

## gender

In [196]:
path_gender = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-collection_data/00-id_data/03-gender/'
all_gender = []
for i in listdir(path_gender):
    try:
        data = pd.read_json(path_gender + i)
        all_gender.append(data)
    except:
        print(i)

mapping_gender64.json


In [197]:
data_gender = pd.concat(all_gender)

In [349]:
data_gender.gender.unique()

array(['male', 'female'], dtype=object)

In [351]:
with open(path_gender + 'mapping_gender64.json') as f:
    data64 = json.load(f)

In [403]:
temp64 = pd.DataFrame({'nombre_completo':list(data64.keys()), 
             'gender':list(data64.values())})

In [363]:
len(temp64)

500

In [387]:
id64 = id_propietarios[id_propietarios['legislatura'] == 64][['id_legislador', 'nombre_completo']]

In [405]:
import fuzzymatcher
matched_results = fuzzymatcher.fuzzy_left_join(temp64,
                                            id64,
                                            ['nombre_completo'],
                                            ['nombre_completo'],
                                            left_id_col='nombre_completo',
                                            right_id_col='nombre_completo')
matched_results = matched_results.reset_index()

In [421]:
clean64 = []
for i in range(0, len(matched_results)):
    score = matched_results.loc[i]['best_match_score']
    if score < .20:
        clean64.append(pd.DataFrame({'id_legislador':[matched_results.loc[i]['id_legislador']], 
                       'gender':[np.nan], 
                       'nombre_completo':[matched_results.loc[i]['nombre_completo_right']]}))
    else:
        clean64.append(pd.DataFrame({'id_legislador':[matched_results.loc[i]['id_legislador']], 
                       'gender':[matched_results.loc[i]['gender']], 
                       'nombre_completo':[matched_results.loc[i]['nombre_completo_right']]}))

In [446]:
clean64_gender = pd.concat(clean64).reset_index().drop(['index'], axis = 1)

In [447]:
clean64_gender[clean64_gender['gender'].isna()].nombre_completo.unique()

array(['González Márquez Karen Michel',
       'Mora Ríos María Teresa Rebeca Rosa',
       'Diaz García María Elizabeth', 'Martínez González Laura',
       'Méndez De La Luz Dauzón Dulce María', 'Lizeth Sanchez Garcia',
       'Báez Ruiz Claudia', 'López Chávez Maria Beatriz',
       'Bautista Peláez Maria del Carmen', 'Hidalgo Ponce Javier Ariel',
       'Rosas Uribe Erika Mariana', 'Guzmán Valdéz Edgar',
       'Juan Carlos Irma', 'Zarzosa Sánchez Eduardo',
       'Contreras Montes Gustavo', 'Alvarado Moreno María Guillermina',
       'De Jesús Jiménez Lucio', 'Alfonso Ramirez Cuellar',
       'Pérez Hernández Sergio', 'Montes Nieves Jorge Luis',
       'Romero Herrera Jorge', 'Baldenebro Arredondo Manuel de Jesus',
       'Bautista Villegas Oscar', 'Tagle Martínez Martha',
       'Medina Herrera Benito', 'Bautista Bravo Juan Angel',
       'Merlín García María del Rosario', 'Jose Jaime Montes Salas',
       'Vazquez Vazquez Alfredo', 'Mata Carrasco Mario'], dtype=object)

In [448]:
def clean_missing(gender, x):
    if pd.isna(gender):
        if x in ['Hidalgo Ponce Javier Ariel','Guzmán Valdéz Edgar',
                'Juan Carlos Irma', 'Zarzosa Sánchez Eduardo',
           'Contreras Montes Gustavo','De Jesús Jiménez Lucio', 'Alfonso Ramirez Cuellar',
           'Pérez Hernández Sergio', 'Montes Nieves Jorge Luis',
           'Romero Herrera Jorge', 'Baldenebro Arredondo Manuel de Jesus',
           'Bautista Villegas Oscar','Medina Herrera Benito', 'Bautista Bravo Juan Angel','Jose Jaime Montes Salas',
           'Vazquez Vazquez Alfredo', 'Mata Carrasco Mario']:
            return('male')
        else:
            return('female')
    else:
        return(gender)

In [449]:
clean64_gender['gender'] = clean64_gender.apply(lambda x: clean_missing(x['gender'], 
                                                                       x['nombre_completo']), axis = 1)

In [450]:
clean64_gender = clean64_gender.rename(columns = {'nombre_completo':'name_legislador'})[['id_legislador', 
                                                                       'name_legislador', 
                                                                       'gender']]

In [461]:
data_gender_temp = pd.concat([data_gender, clean64_gender])

In [464]:
data_gender_temp.gender.unique()

array(['male', 'female'], dtype=object)

In [463]:
data_gender_temp['gender'] = np.where(data_gender_temp['gender'] == 'M', 'female', data_gender_temp['gender'])
data_gender_temp['gender'] = np.where(data_gender_temp['gender'] == 'H', 'male', data_gender_temp['gender'])

In [456]:
id_propietarios_prep = id_propietarios[['legislatura', 'id_legislador', 'nombre_completo']]

In [466]:
gender_clean = pd.merge(left = id_propietarios_prep, 
         right = data_gender_temp, 
         on = 'id_legislador', 
         how = 'left')

In [473]:
gender_clean[gender_clean['gender'].isna()].nombre_completo.unique()

array(['Fernández Noroña José Gerardo Rodolfo', 'González Yáñez Óscar',
       'Ibarra Pedroza Juan Enrique',
       'Martínez y Hernández Ifigenia Martha', 'Muñoz Ledo Porfirio',
       'Reyes Sahagún Teresa Guadalupe', ' Anel Patricia Nava Pérez',
       'Vázquez González Pedro', 'Kahwagi Macari Jorge Antonio',
       'Martínez Peña Elsa María', ' Reyes S. Tamez Guerra',
       'Orozco Torres Norma Leticia', 'Pérez de Alva Blanco Roberto',
       'Pinedo Alonso Cora', 'Torre Canales María del Pilar',
       'Villarreal Benassini Karla Daniella', 'Álvarez Cisneros Jaime',
       'Arizmendi Campos Laura (LICENCIA)',
       'García Almanza María Guadalupe', 'Gertz Manero Alejandro',
       'Jiménez León Pedro', 'Ochoa Mejía Ma. Teresa Rosaura',
       'Serrano Jiménez Emilio', 'Torres Abarca O. Magdalena',
       'Vázquez Aguilar Jaime Arturo', 'Abdala Carmona Yahleel',
       'Abdala Saad Antonio Tarek', 'Aguilar Yunes Marco Antonio',
       'Aguirre Romero Andres', 'Alcantara Rojas Ma

In [476]:
def clean_missing(gender, x):
    if pd.isna(gender):
        if x in ['Martínez y Hernández Ifigenia Martha',  'Martinez Guzman Norma Edith',
       'Perea Santos Ana Guadalupe','Plascencia Pacheco Laura Nereida','Flor Ivone Morales Miranda', 
                 'Miroslava Carrillo Martinez','Xochitl Nashielly Zagal Ramirez','Lorena Cuellar Cisneros',
                 'Adriana Aguilar Vazquez', 'Alemán Hernández Nohemí','Ana Gabriela Guevara Espinoza',
                 'Alma Marina Vitela Rodriguez', 'Marina del Pilar Avila Olmeda', 'Beatriz Manrique Guevara',
                 'Jorgina Gaxiola Lezama', 'Daniela De Los Santos Torres','Ramirez Nachis Rosa Alba',
                  'Gonzalez Luna Josefina','Ocampo Bedolla Maria Eugenia','Garcia Soberanes Cynthia Gissel',
       'Palomares Ramirez Elvia Graciela', 'Reyes Avila Angelica', 'Soralla Banuelos de la Torre',
       'Sanchez Ruiz Karina', 'Torres Sandoval Melissa',
                 'Villanueva Huerta Claudia','Veronica Delgadillo Garcia','Mercado Sanchez Ma. Victoria',
       'Gonzalez Torres Sofia', 'Guevara Jimenez Yaret Adriana','Limon Garcia Lia','Hauffen Torres Angie Dennisse', 
                 'Hernandez Colin Norma Xochitl','Hernandez Correa Guadalupe','Ochoa Avalos Maria Candelaria',
                 'Garcia Perez Maria',  'Flores Sonduk Lluvia','Hurtado Arana Karen','Meza Guzman Lucia Virginia',
                  'Montiel Reyes Ariadna','Ramos Ramirez Blandina','Arzaluz Alonso Alma Lucia', 'Avila Serna Maria',
       'Sandra Luz Falcon Venegas','Villa Gonzalez Concepcion','Canales Suarez Paloma','Chavarrias Barajas Eloisa',
       'Nahle Garcia Norma Rocio',  'Corona Valdes Lorena','Cuenca Ayala Sharon Maria Teresa',
                 'Romero Castillo Maria Guadalupe Cecilia','Talavera Hernandez Maria Eloisa',
                 'Garcia Jimenez Cuitlahuac', 'Godoy Ramos Ernestina', 'Gomez Alvarez Delfina',
                 'Ramirez Peralta Karen Orney','Saucedo Reyes Araceli', 'Soto Gonzalez Cecilia Guadalupe',
                 'Miriam Tinoco Soto','Aceves Pastrana Patricia Elena','Baron Ortiz Natalia Karina',
                  'Barrientos Pantoja Alicia','Cardenas Mariscal Maria Antonia','Galico Felix Diaz Sara Paola',
       'Carrera Hernandez Ana Leticia', 'Chavez Garcia Maria', 'Contreras Julian Maricela',
       'Cuata Dominguez Blanca Margarita', 'Esquivel Valdes Laura Beatriz', 
                 'Madrigal Sanchez Araceli','Parra Alvarez Evelyn','Pool Pech Isaura Ivanova',
                 'Garcia Bravo Maria Cristina Teresa','Gaytan Hernandez Cristina Ismene',
       'Tamariz Garcia Ximena','Valles Olvera Elva Lidia','Aragon Castillo Hortensia',
       'Arguijo Herrera Tania Victoria', 'Beltran Reyes Maria Luisa', 'Lopez Lopez Irma Rebeca',
       'Vargas Barcena Marisol', 'Vega Olivas Nadia Haydee','Briones Perez Erika Irazema',
       'Castelan Mondragon Maria Elida','Cruz Molina Eva Florinda',
       'Velazquez Valdez Brenda', 'Mayra Angelica Enriquez Vanderkam',
       'Acosta Naranjo Guadalupe',
                 'Hernandez Ramos Minerva','Sanchez Carrillo Patricia',
       'Sanchez Juarez Claudia', 'Sanchez Meza Maria Luisa',
       'Schemelensky Castro Ingrid Krasopani (LICENCIA)',
                 'Huerta Villegas Genoveva','Maria Guadalupe Murguia Gutierrez', 'Jimenez Marquez Martha Cristina',
                 'Marquez Zapata Nelly del Carmen','Moya Marin Angelica (LICENCIA)', 'Munoz Cervantes Arlette Ivette',
       'Nava Mouett Jacqueline','Osuna Carranco Karla Karina', 'Ovando Reazola Janette (LICENCIA)','Padilla Avila Karina',
                 'Paniagua Figueroa Luz Argelia', 'Gina Andrea Cruz Blackledge','Ramirez Ramos Gabriela', 
                 'Reynoso Sanchez Alejandra Noemi', 'Rodriguez Aguirre Maria de los Angeles','Rodriguez Rubio Maria del Rosario',
                 
                 
                 'Gonzalez Suastegui Guadalupe', 'Gutierrez Campos Alejandra',
                 'Elizarraraz Sandoval Adriana', 'Patricia Garcia Garcia', 'Garcia Escobar Lilia Arminda',
                 'Reyes Sahagún Teresa Guadalupe', 'Madrigal Mendez Liliana Ivette',
                 'Maya Pineda Maria Isabel', 'Mazari Espin Rosalina','Ivonne Aracelly Ortega Pacheco',
                 'Aleman Olvera Emma Margarita', 'Arambula Melendez Mariana',
       'Alfaro Garcia Lorena del Carmen', 'Amparano Gamez Leticia','Bolio Pinelo Kathia Maria',
                  'Viggiano Austria Alma Carolina','Villa Trujillo Edith','Zapata Lucero Ana Georgina',
       'Zepahua Garcia Lillian', 'Monica Rodriguez Della Vecchia', 'Agundis Estrada Maria Veronica',
                 'Mondragon Orozco Maria Angelica', 'Munoz Sanchez Rosalinda', 'Ortiz Lanz Adriana del Pilar',
       'Ma. Marcela Gonzalez Salas y Petricioli','Oropeza Olguin Nora Liliana','Osorno Belmont Susana', 'Oyervides Valdez Maria Guadalupe',
       'Carolina Monroy Del Mazo', 'Pinete Vargas Maria del Carmen','Ponce Beltran Esthela de Jesus',
                 'Quinones Cornejo Maria De La Paz','Real Salinas Dora Elena','Sandra Mendez Hernandez',
       'Renteria Medina Flor Estela','Rivera Carbajal Silvia', 'Rocha Aguilar Yulma','Salazar Espinosa Heidi',
                 'Sanchez Arredondo Nancy Guadalupe', 'Sanchez Coronel Cristina','Sandoval Martinez Maria Soledad',
                 'Scherman Leano Maria Esther de Jesus',  'Tamayo Morales Martha Sofia (LICENCIA)', 'Tannos Cruz Yarith',
       'Terrazas Porras Adriana', 'Tiscareno Agoitia Ruth Noemi','Velez Nunez Beatriz',
                 'Sobreyra Santos Maria Monserrath',
       'Maricela Serrano Hernandez', 'Rodriguez Hernandez Erika Araceli',
                 ' Anel Patricia Nava Pérez', 'Martínez Peña Elsa María', 'Orozco Torres Norma Leticia',
                 'Torre Canales María del Pilar','Villarreal Benassini Karla Daniella', 'Álvarez Cisneros Jaime',
       'Arizmendi Campos Laura (LICENCIA)','Ochoa Mejía Ma. Teresa Rosaura','Torres Abarca O. Magdalena',
       'García Almanza María Guadalupe','Alcantara Rojas Maria Guadalupe','Alpizar Carrillo Lucely Del Perpetuo Socorro',
                 'Alvarado Varela Edith Anabel', 'Anaya Mota Claudia Edith','Arcos Velazquez Montserrat Alicia',
                 'Arroyo Bello Erika Lorena',  'Barrientos Cano Laura Mitzi','Benitez Tiburcio Mariana',
                 'Bernal Casique Iveth', 'Sylvana Beltrones Sanchez','Guerrero Esquivel Araceli',
       'Boone Godoy Ana Maria', 'Yolanda De la Torre Valdez','De Leon Maza Sofia Del Sagrario',
       'Diaz Montoya Rocio','Maria Del Rocio Rebollo Mendoza','Esquivel Hernandez Olga Maria',
                 'Etcheverry Aranda Azul','Gamboa Martinez Alicia Guadalupe','Guillen Vicente Mercedes Del Carmen',
                 'Gutierrez Ramirez Virginia Nallely', 'Guzman Lagunes Noemi Zoila','Leyva Garcia Leydi Fabiola',
       'Maria Veronica Munoz Parra', 'Lopez Velasco Edith Yolanda','Luna Munguia Alma Lilia',
       'Guzman Vazquez Laura Valeria','Hernandez Madrid Maria Gloria', 'Ibarra Rangel Miriam Dennis', 
                 
       'Felix Niebla Gloria Himelda', 'Fernandez Marquez Julieta','Rosa Elena Millan Bueno', 
                 'Gonzalez Calderon Martha Hilda','Guerrero Coronado Delia',
       'Botello Santibanez Maria Barbara','Bugarin Rodriguez Jasmine Maria','Camargo Felix Maria Esther Guadalupe',
                 'Cavazos Cavazos Juana Aurora','Covarrubias Anaya Martha Lorena',
       'Ceja Garcia Xitlalic', 'Cordova Moran Hersilia Onfalia Adamina', 'Corella Platt Susana',
                ]:
            return('female')
        else:
            return('male')
    else:
        return(gender)

In [477]:
gender_clean.nombre_completo

,legislatura,id_legislador,nombre_completo,name_legislador,gender
0,60,e7f3daa8-fe1f-11ea-95ca-acde48001122,Acosta Dávila Constantino,Acosta Dávila Constantino,male
1,60,e82a705e-fe1f-11ea-95ca-acde48001122,Efraín Arizmendi Uribe,Efraín Arizmendi Uribe,male
2,60,e85d8a34-fe1f-11ea-95ca-acde48001122,Alcaraz Hernández Alma Edwviges (LICENCIA),Alcaraz Hernández Alma Edwviges (LICENCIA),female
3,60,e88f3462-fe1f-11ea-95ca-acde48001122,Álvarez Bernal María Elena,Álvarez Bernal María Elena,female
4,60,e8c1e6c8-fe1f-11ea-95ca-acde48001122,Amezola Fonceca Gerardo,Amezola Fonceca Gerardo,male
...,...,...,...,...,...
3026,64,27cc57ec-fd10-11ea-83d8-acde48001122,Vidal Peniche Jesús Carlos,Vidal Peniche Jesús Carlos,male
3027,64,28723afe-fd10-11ea-83d8-acde48001122,Villafuerte Zavala Lilia,Villafuerte Zavala Lilia,female
3028,64,28fa506a-fd10-11ea-83d8-acde48001122,Morales Vázquez Carlos Alberto,Morales Vázquez Carlos Alberto,male
3029,64,29eb5122-fd10-11ea-83d8-acde48001122,Pérez Rivera Evaristo Lenin,Pérez Rivera Evaristo Lenin,male


In [478]:
gender_clean['gender'] = gender_clean.apply(lambda x: clean_missing(x['gender'], 
                                                                       x['nombre_completo']), axis = 1)

In [485]:
gender_clean = gender_clean[['legislatura', 'id_legislador', 'nombre_completo', 'gender']]

In [497]:
gender_clean = gender_clean.drop_duplicates('id_legislador', keep = 'first')

In [499]:
gender_clean[gender_clean['gender'].isna()]

,legislatura,id_legislador,nombre_completo,gender


## Career Track

In [218]:
path_career = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-collection_data/00-id_data/01-career_track/'

all_career = []
for i in listdir(path_career):
    try:
        data = pd.read_json(path_career + i)
        all_career.append(data)
    except:
        print(i)
        
data_career = pd.concat(all_career)
escolaridad = data_career[data_career['area_experiencia'] == 'ESCOLARIDAD']

In [219]:
escolaridad['escolaridad'] = escolaridad.apply(lambda x: education(x['grado']), axis = 1)
escolaridad['escolaridad_num'] = escolaridad.apply(lambda x: education_num(x['escolaridad']), axis = 1)

In [220]:
id_legisladores = list(id_propietarios.id_legislador)

In [233]:
escolaridad_data = []
for i in tqdm(id_legisladores):
    temp = escolaridad[escolaridad['id_legislador'] == i]
    if len(temp) == 0:
        escolaridad_data.append(pd.DataFrame({'id_legislador':[i], 
                                             'grado_escolaridad':[0]}))
    else:
        grado = max(temp.escolaridad_num)
        escolaridad_data.append(pd.DataFrame({'id_legislador':[i], 
                                             'grado_escolaridad':[grado]}))
    

100%|██████████| 2500/2500 [00:03<00:00, 696.25it/s]


In [234]:
data_escolaridad = pd.concat(escolaridad_data)

In [235]:
missing_data = data_escolaridad[data_escolaridad['grado_escolaridad'].isin([0, 'missing'])]

In [236]:
data_career[data_career['id_legislador'].isin(list(missing_data.id_legislador))][data_career[data_career['id_legislador'].isin(list(missing_data.id_legislador))]['area_experiencia'] == 'ESCOLARIDAD'].grado.unique()

array([], dtype=object)

In [242]:
data_escolaridad

,id_legislador,grado_escolaridad
0,e7f3daa8-fe1f-11ea-95ca-acde48001122,2
0,e82a705e-fe1f-11ea-95ca-acde48001122,0
0,e85d8a34-fe1f-11ea-95ca-acde48001122,3
0,e88f3462-fe1f-11ea-95ca-acde48001122,3
0,e8c1e6c8-fe1f-11ea-95ca-acde48001122,2
...,...,...
0,27cc57ec-fd10-11ea-83d8-acde48001122,2
0,28723afe-fd10-11ea-83d8-acde48001122,2
0,28fa506a-fd10-11ea-83d8-acde48001122,3
0,29eb5122-fd10-11ea-83d8-acde48001122,2


In [253]:
experience = data_career[data_career['area_experiencia'] != 'ESCOLARIDAD']

In [307]:
def obtain_career_track(x, grado, institucion):
    if x in ['ACTIVIDAD EMPRESARIAL', 'INICIATIVA PRIVADA', 
            'Actividad Empresarial', 'ACTIVIDADES EMPRESARIALES', 'ACTIVIDAD EMPRESARIAL']:
        return('experiencia sector privado')
    elif x in ['TRAYECTORIA POLÍTICA', 'CARGOS EN LEGISLATURAS LOCALES O FEDERALES', 'CARGOS DE ELECCIÓN POPULAR', 
              'EXPERIENCIA LEGISLATIVA', 'ADMINISTRACIÓN PÚBLICA FEDERAL']:
        return('experiencia politica')
    elif x in ['ADMINISTRACIÓN PÚBLICA LOCAL']:
        return('administracion publica')
    elif x in ['ACTIVIDADES DOCENTES', 'PUBLICACIONES']:
        return('docencia')
    elif x in ['ORGANISMOS INTERNACIONALES', 'ASOCIACIONES A LAS QUE PERTENECE']:
        return('asociacion civil')
    elif x in ['LOGROS DEPORTIVOS MÁS DESTACADOS']:
        return('carrera deportiva')
    else:
        return(clean_missing(grado, institucion))

In [308]:
def clean_missing(x, institucion):
    if x in ['Vicepresidente de Desarrollo de Nuevos Negocios LATAM', 
            'Vicepresidente de Desarrollo de Nuevos Negocios',
       'Director Ejecutivo de Inversiones Internacionales',
       'Director para LATAM', 'Vicepresidente',
       'Jefe de Departamento de la Coordinación General',
       'Jefe de Departamento de Servicios Periciales',
       'Secretario Particular del Coordinador General', 'Asesor',  'Invitado', 'Ponente', 
            'Coordinadora de Módulo','Director General', 'Director Médico',
       'Campaña de limpieza en barranca en la colonia "El Ermitaño"', 'Propietario', 
            'Jefe de Sistemas', 'Control de obra', 'Fundador', 'Presidente',
            'Enlace de alta responsabilidad', 'Médico',
       'Médico encargado del Servicio de Urgencias de fin de semana',
       'Médico encargado del Servicio de Urgencias Nocturna',
       'Médico encargada del Servicio Médico']:
        return('experiencia sector privado')
    
    
    elif x in ['Secretario General de la Agrupación Política Nacional "Movimiento Nacional de Expresión Política"', 
              'Candidata electa a la Diputación Federal por el Distrito 1, Chiapas',
       'Diputada(o) Federal ,', 'Senador(a) de la República ,',
       'Diputada(o) Constituyente ,',
       'Secretaria de la Juventud en el estado de Quintana Roo',
       'Integrante del Consejo Político Nacional',
       'Secretaria de la Mujer en el estado de Quintana Roo',
       'Representante estatal',
       'Enlace del CEN para el 4º Distrito Federal, estado de Tabasco','Enlace del Distrito 1, Chiapas',
               'Enlace del CEN para el 4º Distrito Federal, estado de Tabasco',
       'Coordinador de Organización del Distrito Fedeal 4. estado de Tabasco',
              'Candidato a diputación federal','Senador(a) de la República Propietario, PAN',
       'Candidato a la Presidencia Municipal de Zacatecas,Zac','Delegado Municipal',
              'Consejero del Poder Legislativo en el Instituto Federal Electoral',
       'Vocero Nacional del Partido Verde Ecologista de México',
       'Directora de Promoción y Fomento de Vivienda', 'Vocero nacional',
       'Dirigente nacional']:
        return('experiencia politica')
    
    
    elif x in ['Coordinador de Auditor{ia', 'Subcontralor', 'Director General de Vinculación Municipal de la Secretaría',
       'Director General de Atención a Municipios de la Secretaría',
       'Subsecretario de Vinculación', 'Coordinador de Asesores',
       'Presidente del Comité Ejecutivo Estatal', 'Presidenta del Comité Municipal', 'Secretaria General Estatal', 
              'Integrante del Consejo Consultivo en Morelos',
       'Integrante del Consejo Consultivo Estatal en Morelos', 'Enlace del Distrito 1, Chiapas', 
              'Jefe de trafico', 'Administrativo',
       'Secretario de Desarrollo Social','Coordinador General del Comité de Planeación para el Desarrollo del estado de Guerrero',
       'Coordinador general de Fortalecimiento Municipal',
       'Enlace Distrital', 'Subdirectora de Atención a Grupos Vulnerables',
       'JUD de Capacitación', 'Consejero Nacional',
       'Secretario de Acción de Gobierno CEN',
       'Coordinador de Asuntos Internacionales CEN',
       'Miembro de la Comisión Permanente Nacional', 'Consejero', 'Enlace de la Secretaría General del Comité Ejecutivo Nacional en los estados de la Segunda Circunscripción',
       'Consejero nacional',
       'Presidente de la Comisión de Orden del Consejo nacional',
       'Enlace de la Secretaría General del Comité Ejecutivo Nacional en los estados de la Primera Circunscripción',
       'Integrante de la Comisión Permanente Estatal',
       'Integrante de la Comisión Permanente Nacional',
       'Integrante de la Comisión Nacional Organizadora del CEN',
       'Miembro de la Comisión Permanente Nacional del CEN',
       'Coordinadora Nacional de desarrollo sustentable del CEN',
       'Consejera Nacional',
       'Coordinadora de medio ambiente en la campaña con la coacilicón "Por México al frente"',
       'Secretario General CDE', 'Secretaria del PPM',
       'Consejera Estatal', 'Presidente del Coimité Directivo Local',
       'Integrante de la Comisión Permanente Regional',
       'Consejera Estatal Hidalgo', 'Consejera Nacional por Hidalgo',
       'Miembro del Comité Directivo Estatal-Hidalgo',
       'Secretario de Acción Juvenil Sahuayo',
       'Presidente del Comité Directivo Municipal', 'Consejero Estatal','Secretario de Elecciones', 'Secretario General',
       'Presidente del Comité Directivo Estatal en el Estado de México',
       'Secretario de Organización del CEN',
       'Secretario Regional del CEN', 'Consejera Politica Municipal',
       'Secretaria General del Frente Juvenil Revolucionario Estatal de Querétaro',
       'Secretaria General del Comité Directivo Municipal de Querétaro',
       'Secretaria Jurídica del Comité Ejecutivo Nacional de la CNOP',
       'Consejera Política Nacional',
       'Coordinadora de Legisladores de la CNOP',
       'Secretaria General del Comité Directivo Municipal en Guadalupe, Nuevo Léon',
       'Secretaria de Organización del CDE en Nuevo León',
       'Coordinadora Regional de campaña del candidato a la Presidencia de la República en Coahuila, Tamaulipas y San Luis Potosí',
       'Secretaria General del Comité Directivo Estatal, Nuevo León',
       'Candidata a Gobernadora del Estado de Nuevo León',
       'Vocera del CEN en el Senado de la República',
       'Coordinadora Nacional de Equidad e Inclusión en la Campaña Presidencial "Todos por México"',
       'Subdirector de Administración Cautelar',
       'Director de Apoyos Finacieros', 'Subsecretario de Egresos',
       'Consejero Político Nacional',
       'Delegado del CEN del PRI EN 15 Estados', 'Secretario',
       'Subsecretario General de Gobiermo',
       'Secretario de Acción Electoral del CEN',
       'Secretario General del CEN', 'Directora de Contacto Ciudadano',
       'Coordinadora Regional',
       'Presidenta Estatal ONMPRI, San Luis Potosí',
       'Secretaria del Comité Directivo Estatal',
       'Presidenta del Comité Directivo Municipal de la Capital',
       'Consejero Político Municipal', 'Consejero Político Estatal',
       'Secretario General del Movimiento Territorial',
       'Subdirector Médico',
       'Integrante de la Comisión Coordinadora Nacional', 'Representante Suplente IFE/INE',
       'Integrante de la Comisión Operativa Nacional',
       'Director de la Juventud','Secretaria de Acuerdos Consejo Nacional',
       'Representante Suplente IFE/INE',
       'Integrante de la Comisión Operativa Nacional',
       'Director de la Juventud', 'Delegado Municipal',
       'Director de Desarrollo Social', 'Embajadora de campaña',
       'Locutora', 'Directora del Programa Mundo Maya',
       'Directora de Proyectos Especiales del Ejecutivo', 'Presidenta',
       'Directora de Relaciones Interinstitucionales',
       'Directora del Sub-Centro del C-4 Zona Norte',
       'Subsecretaria de Coordinación de Programas Sociales Zona Norte',
       'Candidata a Diputada Local por el Dtto. VII, Agua Prieta, Sonora',
       'Secretaria Técnica del Consejo Político CEN de la Red de Jóvenes X México',
       'Consejera Nacional CEN']:
        return('administracion publica')
    
    
    elif x in ['Conferencista', 'Docente', 'Docente frente a grupo']:
        return('docencia')
    
    elif x in [ 'Boxeador Profesional',
       'Campeón de los Supergallo',
       'Campeón de peso supergallo de la Organización Mundial de Boxeo (OMB).',
       'Campeón Pluma Interino del CMB.', 'Promotor de Boxeo',
       'Campeón interino de Superligero del CMB.', 'Entrenador de Boxeo',
       'Miembro del Salón de la Fama de Las Vegas.']:
        return('carrera deportiva')
    
    
    else:
        return(clean_institucion(institucion)) 

In [305]:
def clean_institucion(institucion):
    if institucion in ['Star Technology and science', 'Terrafix México', 
                      'VYV Consultoría y Soluciones Empresariales', 
                      'Grupo Financiero Banorte', 'Grupo Oro', 'Agrupación musical Piluso']:
        return('experiencia sector privado')
    
    elif institucion in [ 'Secretaría de Desarrollo Urbano yObras Públicas',
                        'Coordinaciones de Procuración y Seguridad Pública',
       'Secretaría de Salud. Sinaloa','DIF Guadalajara, Jalisco', 'CONADE', 
                        'Subsecretaría dePrevención del Delito. SEGOB']:
        return('administracion publica')
    
    elif institucion in ['PVEM', 'PAN','Gobierno del estado de Guerrero', 'LVI Legislatura',
       'LVII Legislatura','Morena', 'LXI Legislatura', 'No indica Legislatura', 'PT','PRI']:
        return('experiencia politica')
    
    elif institucion in [ 'Instituto de las Mujeres del Distrito Federal Sistema de Centros Integrales de Apoyo a la Mujer',
                        'Por un México sin Hambre']:
        return('asociacion civil')
    
    
    elif institucion in ['Foro: Bases antropológicas', 'Asignatura: Estadística', 
                        'Asignatura: Teoremas de la Comunicación Género Periodístico Radio']:
        return('docencia')

In [309]:
experience['career_track'] = experience.apply(lambda x: obtain_career_track(x['area_experiencia'], 
                                                                           x['grado'], 
                                                                           x['institucion']), axis = 1)

In [322]:
experience.career_track.unique()

array(['experiencia politica', 'experiencia sector privado',
       'administracion publica', 'asociacion civil', 'docencia',
       'carrera deportiva', None], dtype=object)

In [335]:
def obtain_dummy(type_carrer, x):
    if type_carrer in x:
        return([1])
    else:
        return([0])

In [337]:
career_data = []
for i in tqdm(id_legisladores):
    temp = experience[experience['id_legislador'] == i]
    if len(temp) == 0:
        career_data.append(pd.DataFrame({'id_legislador':[i], 
                                        'exp_pol':[0], 
                                        'exp_sec_privado':[0], 
                                        'admon_publica':[0], 
                                        'asoc_civil':[0], 
                                        'docencia':[0], 
                                        'carrera_deportiva':[0]}))
    else:
        careers = list(temp.career_track.unique())
        career_data.append(pd.DataFrame({'id_legislador':[i], 
                                        'exp_pol':obtain_dummy('experiencia politica',careers), 
                                        'exp_sec_privado':obtain_dummy('experiencia sector privado',careers),
                                        'admon_publica':obtain_dummy('administracion publica',careers),
                                        'asoc_civil':obtain_dummy('asociacion civil',careers),
                                        'docencia':obtain_dummy('docencia',careers),
                                        'carrera_deportiva':obtain_dummy('carrera deportiva',careers)
                                        }))

100%|██████████| 2500/2500 [00:07<00:00, 323.88it/s]


In [340]:
carrer_track_data = pd.concat(career_data)

In [500]:
demographics = pd.merge(left = gender_clean, 
                       right = data_escolaridad, 
                       on = 'id_legislador', 
                       how = 'left')

demographics = pd.merge(left = demographics, 
                        right = carrer_track_data, 
                        on = 'id_legislador', 
                       how = 'left')

In [502]:
demographics.to_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-collection_data/09-controls_reg/controls_reg.xlsx')

In [505]:
demographics.columns

Index(['legislatura', 'id_legislador', 'nombre_completo', 'gender',
       'grado_escolaridad', 'exp_pol', 'exp_sec_privado', 'admon_publica',
       'asoc_civil', 'docencia', 'carrera_deportiva'],
      dtype='object')

In [507]:
demographics.grado_escolaridad.unique()

array([2, 0, 3, 1])